## Erreur inverse - Cas symétrique


Soient $A\in \mathcal{S}_n(\mathbb{R})$ inversible, $b\in\mathbb{R}^n$ et $x\in\mathbb{R}^n$ solution de $$Ax=b.$$

On dispose d'un algorithme permettant la résolution numérique d'un tel problème. L'exécution de celui-ci conduit à une solution numérique $\tilde{x}$, entachée d'erreurs : $$A\tilde{x}\neq b. $$

L'erreur inverse de cet algorithme, en norme de Frobenius, peut être obtenue depuis la solution du problème d'optimisation suivant : $$\min_{\Delta \in \mathcal{C}} \Vert \Delta \Vert^2_F,$$

avec $$\mathcal{C}=\left\{ \Delta \in \mathcal{S}_n(\mathbb{R}) \mbox{ t.q. } (A+\Delta)\tilde{x}=b\right\}.$$

On s'intéresse à un problème plus simple qui consiste à chercher un élément de $\mathcal{C}$, si ce dernier n'est pas vide :

$$\hspace{5cm} (\mathcal{E})\quad \mbox{ Trouver }\Delta \in \mathcal{S}_n(\mathbb{R}) \mbox{ t.q. } (A+\Delta)\tilde{x}=b.$$

En écrivant $$\mathcal{C}=\mathcal{C}_1\bigcap\mathcal{C}_2,$$ avec $$\mathcal{C}_1=\mathcal{S}_n(\mathbb{R}), \quad \mathcal{C}_2=\left\{\Delta \in \mathcal{M}_n(\mathbb{R}) \mbox{ t.q. } (A+\Delta)\tilde{x}=b\right\},$$ on est amenés à chercher un élément sur l'intersection de deux convexes fermés non vides.

**Question 1 :** Justifier que ces deux parties sont convexes, fermées et non vides.


On définit pour cela le problème d'optimisation suivant : 
$$\hspace{5cm} (\mathcal{P})\quad \min_{\Delta \in \mathcal{M}_n(\mathbb{R})} f(\Delta)=\max(d(\Delta,\mathcal{C}_1),d(\Delta,\mathcal{C}_2))$$

avec $d(\Delta,\mathcal{C}_i)$ la distance de $\Delta$ à $\mathcal{C}_i$.

**Question 2 :** On choisit de munir $\mathcal{M}_n(\mathbb{R})$ du produit scalaire $<X,Y>=tr(XY^T)$. On a alors $d(\Delta,\mathcal{C}_i)=\Vert \Delta - \Pi_{\mathcal{C}_i}(\Delta)\Vert_F$, avec $\Vert \Vert_F$ la norme de Frobenius, et $\Pi_{\mathcal{C}_i}(\Delta)$ le projeté de $\Delta$ sur $\mathcal{C}_i$. Donner l'expression analytique des $\Pi_{\mathcal{C}_i}(\Delta)$.

**Question 3 :** Proposer un sous-gradient de $f$ en $\Delta$, $\forall \Delta \in \mathcal{C}$.

**Question 4 :** Résoudre le problème $ (\mathcal{P})$ par l'algorithme du sous-gradient.


In [29]:
## Import de packages
using Plots
using LinearAlgebra

In [96]:
## Initialisation
n = 5

# Construction de A \in Sym(R)
tmp = randn(n, n)
A = (tmp + transpose(tmp)) / 2

# Construction de x \in Rn
x_err = randn(n,)

# Construction de DeltaTrue
tmp2 = randn(n, n)
var = 0.01
Δtrue = var * (tmp2 + transpose(tmp2)) / 2

# Construction (inférance) de b
b = (A + Δtrue) * x_err

5-element Vector{Float64}:
 -1.009922526437535
 -0.05003717443898946
 -0.35834329639075096
  1.1701120779956324
 -0.07057695811137576

In [97]:
# Fonction evaluant le norme de Frobenuis
function normF(X::Matrix, Y::Matrix)
    norm = tr(X * Y')
    return norm
end

normF (generic function with 1 method)

In [98]:
# Fonction évaluant f en x
# f(Δ) = max(Δ(D, ΠC1), d(Δ, ΠC2))
function f(Δ::Matrix)
    return max(d(Δ, ΠC1), d(Δ, ΠC2))
end

function d(Δ::Matrix, Π)
    return norm(Δ - Π(Δ))
end

function ΠC1(Δ::Matrix)
    return (Δ + Δ') / 2
end

function ΠC2(Δ::Matrix)
    return Δ * (I - (x_err * x_err') / norm(x_err, 2)^2) + ((b - A * x_err) * x_err') / norm(x_err, 2)^2
end

ΠC2 (generic function with 1 method)

In [108]:
# Fonction évaluant f en x
# ∂f(D) = ∂d1(D) si (1) ∂d2(D) si (2) 
function ∂f(Δ::Matrix)
    if d(Δ, ΠC1) > d(Δ, ΠC2)
        return ∂d1(Δ)
    else
        return ∂d2(Δ)
    end
end

function ∂d1(Δ::Matrix)
    return (Δ - ΠC1(Δ)) / norm(Δ - ΠC1(Δ))
end

function ∂d2(Δ::Matrix)
    return (Δ - ΠC2(Δ)) / norm(Δ - ΠC2(Δ))
end

∂d2 (generic function with 1 method)

In [143]:
# Algo de sous gradient version notebook
# Choix du pas
pas = 4

# Initialisation
Δ0 = zeros(n, n)
Δk = Δ0
Δbest = Δ0
fbest = +Inf
histo = []
itermax = 50

i = 0
while i < itermax
    i = i + 1
    gk = ∂f(Δk)

    if pas == 1
        Δk -= 1e-2 * gk
    elseif pas == 2
        Δk -= 1 / (100i) * gk
    elseif pas == 3
        Δk -= 1 / (norm(gk, 2) * sqrt(i)) * gk
    elseif pas == 4
        Δk -= f(Δk) / norm(gk, 2)^2 * gk
    end

    if f(Δk) < fbest
        fbest = f(Δk)
        Δbest = Δk
    end
    append!(histo, fbest)
end

plotly();
iter = 1:itermax;
plot(iter, histo, title = "Convergence curve", label = ["f"], lw = 3, ann = [(55, 80, "Sous-gradient simple")])